In [ ]:
import os
import heapq

import ssl
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from IPython.display import SVG, Image
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.sequence import pad_sequences

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

# Скачаем данные и посмотрим на них

Мы будем использовать imdb, подробнее про него можно прочитать тут: https://keras.io/datasets/.

In [ ]:
vocab_size = 10000

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=vocab_size,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

word_2_index = imdb.get_word_index()

In [ ]:
word_2_index = {k:(v+3) for k,v in word_2_index.items()} 
word_2_index["<PAD>"] = 0
word_2_index["<START>"] = 1
word_2_index["<UNK>"] = 2
word_2_index["<UNUSED>"] = 3

index_2_word = {value: key for key, value in word_2_index.items()}

In [ ]:
print('Index "100" to word = {}'.format(index_2_word[100]))
print('Word "could" to index = {}'.format(word_2_index['could']))

In [ ]:
def decode_review(text):
    return ' '.join([index_2_word.get(i, '?') for i in text])

In [ ]:
print(decode_review(x_train[0]))
print('')
print('Label = {}'.format(y_train[0]))

In [ ]:
print('Decoded texts for train set')
x_train_decoded = []
for review in tqdm.tqdm_notebook(x_train):
    decoded_review = decode_review(review)
    x_train_decoded.append(decoded_review)

print('Decoded texts for test set')
x_test_decoded = []
for review in tqdm.tqdm_notebook(x_test):
    decoded_review = decode_review(review)
    x_test_decoded.append(decoded_review)

# Классическая обработка текстов

Для векторизации текстов воспользуемся CountVectorizer, он представляет документ как мешок слов. Можно всячески варировать извлечение признаков (убирать редкие слова, убирать частые слова, убирать слова общей лексики, брать биграмы и т.д.)

In [ ]:
Image('../seminar03/pics/bag_of_words.png', width=600)

In [ ]:
count_vectorizer = CountVectorizer(min_df=1,
                                   ngram_range=(1, 1))

Мы будем работать с эффективным представлением матриц "Compressed Sparse Row Format" (CSR). Почитать о формате можно здесь: https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)

In [ ]:
x_train_bow = count_vectorizer.fit_transform(x_train_decoded)
x_test_bow = count_vectorizer.transform(x_test_decoded)

In [ ]:
print('Shape of train sample in BoW format', x_train_bow.shape)
print('Shape of test sample in BoW format', x_test_bow.shape)

In [ ]:
index_2_word_logreg = {
    v: k
    for k, v in count_vectorizer.vocabulary_.items()
}

### Обучим логистическую регрессию предсказывать тональность отзыва на признаках Bag-of-Words

In [ ]:
model_logreg = LogisticRegression()

In [ ]:
model_logreg.fit(x_train_bow, y_train)

###  Попробуем проинтерпретировать коэффициенты модели

In [ ]:
W = model_logreg.coef_.shape[1]
NUM_WORDS = 10
class_2_function = {'Negative': heapq.nsmallest, 'Positive': heapq.nlargest}

for category, function in class_2_function.items():
    topic_words = [
        index_2_word_logreg[w_num]
        for w_num in function(NUM_WORDS, range(W),
                              key=lambda w: model_logreg.coef_[0, w])
    ]
    print(category)
    print(', '.join(topic_words))

### Оценим качество

In [ ]:
preds_logreg = model_logreg.predict(x_test_bow)

In [ ]:
print('Train accuracy', accuracy_score(
    model_logreg.predict(x_train_bow),
    y_train
))
print('Test accuracy', accuracy_score(preds_logreg, y_test))

### Попробуем TF-IDF

In [ ]:
Image('../seminar03/pics/tfidf.png')

Подробнее про tf-idf можно прочитать здесь: https://ru.wikipedia.org/wiki/TF-IDF

In [ ]:
tf_idf_transformer = TfidfTransformer()

In [ ]:
x_train_tfidf = tf_idf_transformer.fit_transform(x_train_bow)
x_test_tfidf = tf_idf_transformer.transform(x_test_bow)

### Обучим логистическую регрессию предсказывать тональность отзыва на признаках TF-IDF

In [ ]:
model_logreg_tfidf = LogisticRegression()

In [ ]:
model_logreg_tfidf.fit(x_train_tfidf, y_train)

In [ ]:
preds_logreg_tfidf = model_logreg_tfidf.predict(x_test_tfidf)

### Оценим качество

In [ ]:
print('Train accuracy', accuracy_score(
    model_logreg_tfidf.predict(x_train_tfidf),
    y_train
))
print('Test accuracy', accuracy_score(preds_logreg_tfidf, y_test))

# Глубокое обучение в обработке текстов

Код частично взят из примеров TensorFlow: https://www.tensorflow.org/tutorials/keras/basic_text_classification

Нужно привести все тексты к одной длине, чтобы их можно было объединять в батчи для статичных графов. Неплохое сравнение динамических и статичных графов можно найти тут: https://stackoverflow.com/questions/46154189/what-is-the-difference-of-static-computational-graphs-in-tensorflow-and-dynamic.

In [ ]:
x_train_pad = pad_sequences(x_train,
                            value=word_2_index["<PAD>"],
                            padding='post',
                            maxlen=256)

x_test_pad = pad_sequences(x_test,
                           value=word_2_index["<PAD>"],
                           padding='post',
                           maxlen=256)

### Определим модель и обучим ее

Модель будет очень простой, мы вставим Embedding слой, который будет давать каждому слову в тексте сжатую репрезентацию, то есть будет переводить его из размерности vocab_size в размерность 16. Далее GlobalAveragePooling1D слой будет усреднять репрезентации всех слов в тексте. На основе усредненной репрезентации маленький перцептрон будет проводить классификацию.

In [ ]:
Image('pics/matrix_mult_w_one_hot.png')

Установим сиды, чтобы получать воспроизводимые результаты.

In [ ]:
np.random.seed(2)
tf.set_random_seed(2)

In [ ]:
model_dl = Sequential()
model_dl.add(Embedding(vocab_size, 16))
model_dl.add(GlobalAveragePooling1D())
model_dl.add(Dense(units=16, activation='relu'))
model_dl.add(Dropout(0.5))
model_dl.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model_dl.summary()

In [ ]:
SVG(model_to_dot(model_dl).create(prog='dot', format='svg'))

In [ ]:
opt = Adam(lr=0.001)

model_dl.compile(optimizer=opt,
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [ ]:
history = model_dl.fit(x=x_train_pad,
                       y=y_train,
                       epochs=20,
                       batch_size=512,
                       validation_data=(x_test_pad, y_test),
                       verbose=1)

In [ ]:
preds_dl = model_dl.predict(x_test_pad, verbose=1, batch_size=512)
preds_dl_train = model_dl.predict(x_train_pad, verbose=1, batch_size=512)

### Оценим качество

In [ ]:
print('Train accuracy', accuracy_score(y_train, (preds_dl_train>0.5)*1.))
print('Test accuracy', accuracy_score(y_test, (preds_dl>0.5)*1.))

### Посмотрим на динамику обучения

In [ ]:
fig = plt.figure(figsize=(15,7.5))

epochs = range(1, len(history.history['acc']) + 1)

plt.plot(epochs, history.history['acc'], 'r', label='Training loss')
plt.plot(epochs, history.history['val_acc'], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()